In [ ]:
import serial # ステージコントローラ用に使うインタフェース　RS232C
import time
from time import sleep
import visa # パワーメータ用に使うインタフェース

# シリアルポートOPEN、RS232C->USB変換ケーブルを使う。デバイスマネージャでCOMポートを見て、出てきた変換ポートのCOM番号を使う。ここではCOM6だった。
ser = serial.Serial("COM4", baudrate=9600, parity=serial.PARITY_NONE, 
                    bytesize = serial.EIGHTBITS, stopbits = serial.STOPBITS_ONE,
                    timeout = None, xonxoff = 0, rtscts = 0,)

path = 'Data.txt' # データ保存用のファイル名。毎回ここを変える

# パワーメータはUSB接続だが、VISAというインタフェースモジュールを使う。
# visa32.dllのパスを設定する。これは環境によって異なる。これは菅の環境の場合。program files以下を探す。
rm = visa.ResourceManager('C:/Program Files (x86)/IVI Foundation/VISA/WinNT/agvisa/agbin/visa32.dll')

# visaのリソースのリストが出てくる。ここで分かったアドレスを次のopen_resourceで記入する
print(rm.list_resources())

In [ ]:
# 以下は、PM320Eのシリアル番号を含んだアドレス。
dso = rm.open_resource('USB0::0x1313::0x8022::M00512252::INSTR')
# s = dso.query("*IDN?") 機器のIDを聞くクエリ

s1range = 1000  # SPR計測時の角度範囲　400パルス = 1°に対応
step = 100 # SPR計測時の角度ステップ



#------------------ステージ駆動メソッド------------------
# stage1の原点を現在の位置で設定
def defineS1Home():
    cmd = 'R:1' + '\r\n'
    ser.write(cmd.encode())

# stage2の原点を現在の位置で設定
def defineS2Home():
    cmd = 'R:2' + '\r\n'
    ser.write(cmd.encode())

# stage1および2の原点を現在の位置で設定
def defineS1S2Home():
    cmd = 'R:W' + '\r\n'
    ser.write(cmd.encode())

# stage 1ホームへ移動
def s1Home():
    cmd = 'H:1\r\n'
    ser.write(cmd.encode())

# stage 2ホームへ移動
# 原点設定を行っていないため、無限に回り続ける。衝突と故障の原因となり危険なので使用時は要注意すること。
# 止めるときはステージ電源を落とす
def s2Home():
    cmd = 'H:2+\r\n'
    ser.write(cmd.encode())

# stage 2ホームへ移動
# こっちのメソッドは動作する
def s1HomeSigma():
    cmd = 'H:1\r\n'
    ser.write(cmd.encode())
    
# stage 2ホームへ移動
# こっちのメソッドは動作する
def s2HomeSigma():
    cmd = 'H:2\r\n'
    ser.write(cmd.encode())

# stage 1反時計回り（Counter Clock Wise）にpulse分だけ動く、以下の関数も同じ伝です。
def s1ccw(pulse):
    cmd = 'M:1+P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')
        
def s1cw(pulse):
    cmd = 'M:1-P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')
    
def s2ccw(pulse):
    cmd = 'M:2+P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')

def s2cw(pulse):
    cmd = 'M:2-P' + str(pulse) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')


def moveToSprPoint(s1CurrentPulse, s2CurrentPulse, s1SPRPulse):
    INITIAL_SLEEP_TIME = 3.0
    '''
    指定したパルス位置(SPR発生ポイント)へ移動するメソッド

    Parameters
    ----------
    s1CurrentPulse : int
        stage1の現在位置(メソッド駆動時)
    s2CurrentPulse : int
        stage2の現在位置(メソッド駆動時)
    s1SPRPulse : int
        stage1におけるSPR発生位置
        
    Returns
    -------
    s1CurrentPulse : int
        stage1の現在位置(=SPR発生位置)
    s2CurrentPulse : int
        stage2の現在位置(=SPR発生位置)
    '''
    # stage1の移動
    s1PulseToBeMoved = s1CurrentPulse - s1SPRPulse
    s1cw(s1PulseToBeMoved)
    sleep(INITIAL_SLEEP_TIME)
    # stage2の移動
    s2PulseToBeMoved = s1PulseToBeMoved*2
    s2cw(s2PulseToBeMoved)
    sleep(INITIAL_SLEEP_TIME)
    #現在のパルスを更新してリターン
    s1CurrentPulse = s1CurrentPulse - s1PulseToBeMoved
    s2CurrentPulse = s2CurrentPulse - s2PulseToBeMoved
    
    return s1CurrentPulse, s2CurrentPulse

# stage 1とstage 2を反時計回り（Counter Clock Wise）に動かす。stage2は反射計測を考慮して、stage1の倍動く
def s1s2ccw(pulse):
    cmd = 'M:W+P' + str(pulse) + '+P' + str(pulse*2) + '\r\n'
    ser.write(cmd.encode())
    ser.write(b'G\r\n')

# stage1の角度を返す。しかし、serialをopenして一度しか正しい値を返さないので、正確に出すには、毎度ser.close()をしてopenしなおさなくてはならない。
def currentPos(ser):
    s = ''
    while True:
        ser.write(b'Q:\r\n')
        c = ser.read(1)
        # print(c)
        s = s+c.decode('UTF-8')
        if c == b'\n': 
            break
    print(s)
    ssplit = s.split(',')
    return int(ssplit[0])

# SPR計測を行うプログラムのベース。まだ、PM320Eから計測する関数と、ファイルにデータを書き込む部分が未整備。
def sprMeas():
    s1pos = currentpos(ser)
    s1start = s1pos# s1pos : theta1 current pulse position
    s1endpos = s1start + s1range
    
    while s1pos < s1endpos:
        s = dso.query(":POW1:VAL?") # power measurement
        # 計測が完了するまでwaitしておく必要があるだろう
        print(str(s1pos)+'\r\n')
        #with open(path, mode='w') as f:    
        #    f.write(str(pos)+'\r\n')
        # ファイルへのデータ書き込み、大下君に聞いてください。記載するべき事項は、角度(s1pos)+強度値(s)です。
        s1s2ccw(step)
        time.sleep(0.5)
        s1pos = s1pos + step
    ser.close()
    
    
def changeRefPower_String2float(strRefPower):
    '''
    String型の反射光強度をfloat型に変更するメソッド
    ex.) 1.70496365E-04 -> 0.000170496365
    
    Parameters
    ----------
    strRefPower : str
        計測器出力のString型の反射光強度値
        
    Returns
    -------
    floRefPower : float
        float型の反射光強度値
    '''
    if(strRefPower.count("E-02")>0):
        floRefPower = float(strRefPower.rstrip("E-02"))
        floRefPower = floRefPower/(10**2)
        floRefPower = floRefPower*(10**3)
    elif(strRefPower.count("E-03")>0):
        floRefPower = float(strRefPower.rstrip("E-03"))
        floRefPower = floRefPower/(10**3)
        floRefPower = floRefPower*(10**3)        
    elif(strRefPower.count("E-04")>0):
        floRefPower = float(strRefPower.rstrip("E-04"))
        floRefPower = floRefPower/(10**4)
        floRefPower = floRefPower*(10**3)
    elif(strRefPower.count("E-05")>0):
        floRefPower = float(strRefPower.rstrip("E-05"))
        floRefPower = floRefPower/(10**5)
        floRefPower = floRefPower*(10**3)
    elif(strRefPower.count("E-06")>0):
        floRefPower = float(strRefPower.rstrip("E-06"))
        floRefPower = floRefPower/(10**6)
        floRefPower = floRefPower*(10**3)
    return floRefPower

In [ ]:
def initial_SPRmeas():
    '''
    最初にSPR波形をトレース
    そこからθ_SPRをとなる位置データを獲得する
    
    Returns
    -------
    s1CurrentPulse : int
        stage1のSPR発生位置
    s2CurrentPulse : int
        stage2のSPR発生位置   
    '''
    #計測パラメータの設定
    s1CurrentPulse = 0    # Stage1位置データ
    s2CurrentPulse = 0    # Stage2位置データ
    refPower = 0 # 反射光強度
    S1_PALSE_VALUE = 400
    S2_PALSE_VALUE = 800
    INITIAL_SLEEP_TIME = 5.0
    MEASURE_SLEEP_TIME = 0.5
    NUMBER_OF_MEASUREMENTS = 20
    i=0

    #動作Main文
    s1Home() #初期位置へ移動
    sleep(INITIAL_SLEEP_TIME)
    s2HomeSigma()
    sleep(INITIAL_SLEEP_TIME)
    dictThetaPower = {}

    #計測フェーズ
    for i in range(NUMBER_OF_MEASUREMENTS):
        s1ccw(S1_PALSE_VALUE)
        sleep(MEASURE_SLEEP_TIME)
        s2ccw(S2_PALSE_VALUE)
        sleep(MEASURE_SLEEP_TIME)
        
        #値の更新・追加
        s1CurrentPulse += S1_PALSE_VALUE
        s2CurrentPulse += S2_PALSE_VALUE
        refPower = dso.query(":POW1:VAL?").strip()
        refPower = changeRefPower_String2float(refPower)
        dictThetaPower[s1CurrentPulse] = refPower
        print('Stage1：{0}, Stage2：{1}, refPower：{2}'.format(s1CurrentPulse, s2CurrentPulse, refPower))
    s1SPRPulse = min(dictThetaPower, key=dictThetaPower.get) # SPRの発生ポイント
    print("s1 SPR point : {0}".format(s1SPRPulse)) # SPRの発生ポイントを確認
    #print(dictThetaPower)
    
    tupleCurrentPulse = moveToSprPoint(s1CurrentPulse, s2CurrentPulse, s1SPRPulse)
    s1CurrentPulse = tupleCurrentPulse[0]
    s2CurrentPulse = tupleCurrentPulse[1]

    return s1CurrentPulse, s2CurrentPulse


In [ ]:
def followSprPoint():
    #SPR点を追従し続けるプログラム
    """
    MPPT制御をイメージして作成
    2点の反射光強度を計測して、2点から求まる傾きを計算し、
    ・傾きが負ならばパルスを＋する->ccwで回転
    ・傾きが生ならばパルスを-する->cwで回転
    パルス分だけ移動させ、ステージの現在位置を更新する
    """
    #計測パラメータの設定
    countMeasTimes = 0
    s1CurrentPulse = 0    # Stage1位置データ
    s2CurrentPulse = 0    # Stage2位置データ
    refPower = 0 # 反射光強度
    S1_PALSE_VALUE = 400
    S2_PALSE_VALUE = 800
    INITIAL_SLEEP_TIME = 5.0
    MEASURE_SLEEP_TIME = 0.5
    NUMBER_OF_MEASUREMENTS = 20
    listRefPower = [0.0, 0.0] # 2点の反射光強度を格納する
    inclinationOfSprPoint = 0.0 #RefPowerから求まる波形の傾きを格納
    dictThetaPower = {} # 計測値保存用の辞書

    #制御プログラム
    tupleCurrentPulse = initial_SPRmeas()
    s1CurrentPulse = tupleCurrentPulse[0]
    s2CurrentPulse = tupleCurrentPulse[1]
    
    listRefPower[0] = dso.query(":POW1:VAL?").strip()
    listRefPower[0] = changeRefPower_String2float(listRefPower[0])
    while(True):
        countMeasTimes+=1
        #傾きから移動する方向を決定し、回転＆計測
        
        if(inclinationOfSprPoint<0): # 傾きが負ならCCWで回転
            s1ccw(S1_PALSE_VALUE)
            sleep(MEASURE_SLEEP_TIME)
            s1CurrentPulse += S1_PALSE_VALUE
            s2ccw(S2_PALSE_VALUE)
            sleep(MEASURE_SLEEP_TIME)
            s2CurrentPulse += S2_PALSE_VALUE
            listRefPower[1] = dso.query(":POW1:VAL?").strip()
            listRefPower[1] = changeRefPower_String2float(listRefPower[1])

        else:                        # 傾きが正ならCWで回転
            s1cw(S1_PALSE_VALUE)
            sleep(MEASURE_SLEEP_TIME)
            s1CurrentPulse -= S1_PALSE_VALUE
            s2cw(S2_PALSE_VALUE)
            sleep(MEASURE_SLEEP_TIME)
            listRefPower[1] = dso.query(":POW1:VAL?").strip()
            listRefPower[1] = changeRefPower_String2float(listRefPower[1])
        
        print("No.{2},  位置:{0},  計測値:{1}".format(s1CurrentPulse,listRefPower[1],countMeasTimes))
        dictThetaPower[s1CurrentPulse] = listRefPower[1]
        
        #計測値から傾きを算出
        inclinationOfSprPoint = listRefPower[1] - listRefPower[0]
        if(inclinationOfSprPoint>0):
            print("傾きはプラス")
        else:
            print("傾きはマイナス")
        listRefPower[0] =  listRefPower[1]

In [ ]:
followSprPoint()

In [ ]:
#以下　テスト用の記述

In [ ]:
tupleCurrentPulse = initial_SPRmeas()
s1CurrentPulse = tupleCurrentPulse[0]
s2CurrentPulse = tupleCurrentPulse[1]

In [ ]:
print(s1CurrentPulse)
print(s2CurrentPulse)

In [ ]:
s1Home()

In [ ]:
s2HomeSigma()